# Varying diameter (Actually not yet)

In this notebook we will try to create a scaling law to account for a varying propeller diameter as well as a varying number of blade and a varying altitude. Twist law should also vary but for now it will be left as is.

In [ ]:
# This file is part of FAST-OAD_CS23-HE : A framework for rapid Overall Aircraft Design of Hybrid
# Electric Aircraft.
# Copyright (C) 2022 ISAE-SUPAERO

# Import pyVPLM modules/functions
from pyvplm.core.definition import PositiveParameter, PositiveParameterSet
from pyvplm.addon.variablepowerlaw import buckingham_theorem

from stdatm import Atmosphere

altitude_min, altitude_max = 0, 30000 # In feet
rho_min = Atmosphere(altitude_max, altitude_in_feet=True).density
rho_max = Atmosphere(altitude_min, altitude_in_feet=True).density

# Declare physical variables and set (limits have no impact since DOE is not constructed)
t = PositiveParameter("t", [0.01, 46172.16929583475], "N", "trust")
rho = PositiveParameter("rho", [rho_min, rho_max], "kg/m^3", "air density")
n_rot = PositiveParameter("n_rot", [16.66, 33.33], "Hz", "rotational speed")
D = PositiveParameter("D", [2.31], "m", "propeller diameter")
n_blade = PositiveParameter("n_blade", [2, 6], "", "number of propeller blade")
pitch = PositiveParameter("pitch", [0.087, 1.0472], "rad", "propeller pitch @75% of the radius")
PITCH_REF = PositiveParameter("PITCH_REF", [0.087], "rad", "propeller reference pitch @75% of the radius")
v = PositiveParameter("v", [5.0, 191.37333333333333], "m/s", "air speed")
parameter_set = PositiveParameterSet(t, rho, n_rot, D, n_blade, pitch, PITCH_REF, v)

# Define d and lambda_wind to be first elected as repetitive set
parameter_set.first("n_rot", "D", "rho")
parameter_set.latex_render()

# Calculate pi parameters with first repetitive set found in parameters' order
pi_set, _ = buckingham_theorem(parameter_set, False)
pi_set.latex_render()

We'll adjust slightly the defined set here to take something similar to the previous notebook, that is $\pi_2 = \frac{picth}{PITCH_{REF}}$ and $\pi_3 = n_{blade}$

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.core.definition import Constraint, ConstraintSet
from pyvplm.addon.pixdoe import create_const_doe
from pyvplm.addon.variablepowerlaw import (
    declare_func_x_to_pi,
    reduce_parameter_set,
    declare_constraints,
)
from pyvplm.addon.comsoladdon import save_file

reduced_parameter_set, reduced_pi_set = reduce_parameter_set(
    parameter_set, pi_set, elected_output="t"
)
reduced_pi_set.latex_render()
reduced_parameter_set.latex_render()

constraint1 = Constraint("n_rot**2*D**2/4 + v**2<=300.0**2")
constraint2 = Constraint("tan(pitch)<=0.27") # C'est ici, j'ai pas mi l'expression complète mais ça montre le problème
parameters_constraints = declare_constraints(reduced_parameter_set, ConstraintSet(constraint1))

# Save function to translate x parameters to pi parameters
func_x_to_pi = declare_func_x_to_pi(reduced_parameter_set, reduced_pi_set)

results = create_const_doe(
    reduced_parameter_set,
    reduced_pi_set,
    func_x_to_pi,
    whished_size=100,
    parameters_constraints=parameters_constraints,
)
doeX = results[0]

In [ ]:
import numpy as np

n_rot_val = np.array(doeX[:, 0])
d_val = np.array(doeX[:, 1])
n_blade_val = np.array(doeX[:,2])
rho_val = np.array(doeX[:, 3])
pitch_val = np.array(doeX[:, 4])
PITCH_REF_val = np.array(doeX[:, 5])
v_val = np.array(doeX[:, 6])

aoa_75 = pitch_val - np.arctan2(v_val * 2, 0.75 * n_rot_val * d_val * 2.0 * np.pi)

proper_index = np.where(abs(aoa_75 - 0.05) < 0.22)[0]

new_doeX = doeX[proper_index, :]

# Save .txt file compatible with COMSOL
file_name = "./data/doe_fixed_d_var"
save_file(new_doeX, file_name, reduced_parameter_set, is_SI=True)

n_rot_val = np.array(new_doeX[:, 0])
d_val = np.array(new_doeX[:, 1])
n_blade_val = np.array(new_doeX[:, 2])
rho_val = np.array(new_doeX[:, 3])
pitch_val = np.array(new_doeX[:, 4])
PITCH_REF_val = np.array(new_doeX[:, 5])
v_val = np.array(new_doeX[:, 6])

In [ ]:
import os.path as pth
from fastga_he.models.propulsion.components.propulsor.propeller.components.performance_point_d_var import (
    ComputePropellerPointPerformanceDVar,
)
from fastga.command.api import generate_block_analysis

# Define relative path
DATA_FOLDER_PATH = "data"
DATA_FILE = pth.join(DATA_FOLDER_PATH, "input_tbm900_d_var.xml")

# Define the wing primary geometry parameters name as a list
var_inputs = [
    "data:geometry:propeller:average_rpm",
    "data:geometry:propeller:diameter",
    "data:geometry:propeller:blades_number",
    "data:aerodynamics:propeller:point_performance:rho",
    "data:aerodynamics:propeller:point_performance:twist_75",
    "data:aerodynamics:propeller:point_performance:twist_75_ref",
    "data:aerodynamics:propeller:point_performance:speed",
]

# Declare function
compute_propeller = generate_block_analysis(
    ComputePropellerPointPerformanceDVar(),
    var_inputs,
    str(DATA_FILE),
    overwrite=False,
)

In [ ]:
thrust_array = np.zeros_like(n_rot_val)
power_array = np.zeros_like(n_rot_val)
eta_array = np.zeros_like(n_rot_val)

for idx, n_rot_it in enumerate(n_rot_val):
    inputs_dict = {
        "data:geometry:propeller:average_rpm": (n_rot_it, "rps"),
        "data:geometry:propeller:diameter": (d_val[idx], "m"),
        "data:geometry:propeller:blades_number": (n_blade_val[idx], None),
        "data:aerodynamics:propeller:point_performance:rho": (rho_val[idx], "kg/m**3"),
        "data:aerodynamics:propeller:point_performance:twist_75": (pitch_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:twist_75_ref": (PITCH_REF_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:speed": (v_val[idx], "m/s"),
    }
    output_dict = compute_propeller(inputs_dict)
    thrust = output_dict["data:aerodynamics:propeller:point_performance:thrust"][0][0]
    power = output_dict["data:aerodynamics:propeller:point_performance:power"][0][0]
    eta = output_dict["data:aerodynamics:propeller:point_performance:efficiency"][0][0]
    thrust_array[idx] = thrust
    power_array[idx] = power
    eta_array[idx] = eta

proper_result_idx = np.where(abs(eta_array - 0.5) < 0.5)[0]

In [ ]:
thrust_array = thrust_array[proper_result_idx]
power_array = power_array[proper_result_idx]
eta_array = eta_array[proper_result_idx]

n_rot_val = n_rot_val[proper_result_idx]
d_val = d_val[proper_result_idx]
n_blade_val = n_blade_val[proper_result_idx]
rho_val = rho_val[proper_result_idx]
pitch_val = pitch_val[proper_result_idx]
PITCH_REF_val = PITCH_REF_val[proper_result_idx]
v_val = v_val[proper_result_idx]

# Trying $\eta$ as a function of Ct and J

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

import pandas as pd

pi1 = eta_array
pi2 = thrust_array / (rho_val * n_rot_val ** 2 * d_val ** 4)
pi3 = v_val / (n_rot_val * d_val)
pi4 = n_blade_val * 0.121 * 2 / np.pi # Approximately the rotor solidity to be computed properly in the following attempts
# but in this one the aspect ratio of the blade is 1D of the dimater
# Somehow reduced the deviation :O
doe_eta = np.c_[pi1, pi2, pi3, pi4]
doe_eta = pd.DataFrame(doe_eta, columns=["pi1", "pi2", "pi3", "pi4"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_eta.values, elected_pi0="pi1", order=3, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_eta.values, model, choosen_model=20)